In [41]:
!pip install rpy2

  Using cached rpy2-3.5.17.tar.gz (220 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached cffi-1.17.1-cp312-cp312-win_amd64.whl.metadata (1.6 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
Using cached cffi-1.17.1-cp312-cp312-win_amd64.whl (181 kB)
Using cached pycparser-2.22-py3-none-any.whl (117 kB)
  Created wheel for rpy2: filename=rpy2-3.5.17-py3-none-any.whl size=221382 sha256=eb8f95711eccb1eadac5fe49fc5e1dd1fe65abe479de7198741bba1032b62c57
  Stored in directory: c:\users\sahal\appdata\local\pip\cache\wheels\fb\3c\a8\5c2ab28ca010aa26f09174d6ac337141721ee764f7b17c0953
Successfully built rpy2


In [6]:
import pandas as pd
import numpy as np
import pandasql as pql

In [7]:
av_patients = pd.read_csv(r'data\sim_av_patient.csv')
sact_patients = pd.read_csv(r'data\sim_sact_patient.csv')
                          

In [8]:
query = """
SELECT * from av_patients
limit 10;
""" 

In [9]:
result = pql.sqldf(query, locals())
print(result)

   PATIENTID  SEX  LINKNUMBER ETHNICITY DEATHCAUSECODE_1A DEATHCAUSECODE_1B  \
0   10000001    1   810000001         Z              None              None   
1   10000002    1   810000002         A              I259              None   
2   10000003    1   810000003         Z              None              None   
3   10000004    2   810000004         A              None              None   
4   10000005    1   810000005         A              None              None   
5   10000006    2   810000006      None              None              None   
6   10000007    1   810000007         A              None              None   
7   10000008    1   810000008         A              None              None   
8   10000009    1   810000009         A              None              None   
9   10000010    1   810000010         A              None              None   

  DEATHCAUSECODE_1C DEATHCAUSECODE_2 DEATHCAUSECODE_UNDERLYING  \
0              None             None                      None  

In [10]:
#Cohort Selection

query2 = """SELECT * from sact_patients 
limit 10;
"""
result2 = pql.sqldf(query2, locals())
print(result2)


   MERGED_PATIENT_ID  LINK_NUMBER
0           10002576    810002576
1           10007761    810007761
2           10010502    810010502
3           10016805    810016805
4           10018778    810018778
5           10019027    810019027
6           10019936    810019936
7           10025059    810025059
8           10029618    810029618
9           10030668    810030668


In [11]:
query3 = """
select ap.PATIENTID,ap.LINKNUMBER,sp.MERGED_PATIENT_ID
from av_patients as ap
left join sact_patients as sp
on ap.PATIENTID = sp.MERGED_PATIENT_ID
"""
result3 = pql.sqldf(query3, locals())
print(result3)

         PATIENTID  LINKNUMBER  MERGED_PATIENT_ID
0         10000001   810000001                NaN
1         10000002   810000002                NaN
2         10000003   810000003                NaN
3         10000004   810000004                NaN
4         10000005   810000005                NaN
...            ...         ...                ...
2200621  330025052  1130025052        330025052.0
2200622  330025053  1130025053                NaN
2200623  330025054  1130025054                NaN
2200624  330025055  1130025055                NaN
2200625  330025056  1130025056                NaN

[2200626 rows x 3 columns]


In [12]:
queryc =""" Select count(*) from result3
where MERGED_PATIENT_ID is not null;"""
resultc = pql.sqldf(queryc, locals())
print(resultc)

   count(*)
0    366266


In [13]:
#explore the data
av_tumour = pd.read_csv(r'data\sim_av_tumour.csv')
querye2 = """
select * from av_tumour
limit 10;
"""
resulte2 = pql.sqldf(querye2, locals())
print(resulte2)

C:\Users\sahal\AppData\Local\Temp\ipykernel_13348\2253575969.py:2: DtypeWarning: Columns (5,6,8,9,17,18,20,22,24,26) have mixed types. Specify dtype option on import or set low_memory=False.
  av_tumour = pd.read_csv(r'data\sim_av_tumour.csv')


   TUMOURID  PATIENTID DIAGNOSISDATEBEST SITE_ICD10_O2 SITE_ICD10_O2_3CHAR  \
0  10000001   10000001        2017-04-24          C447                 C44   
1  10000002   10000002        2016-10-18          C449                 C44   
2  10000003   10000003        2017-01-13          C447                 C44   
3  10000004   10000004        2015-10-28          C449                 C44   
4  10000005   10000005        2017-10-03          C449                 C44   
5  10000006   10000006        2015-06-04          C440                 C44   
6  10000007   10000007        2014-02-13          C449                 C44   
7  10000008   10000008        2015-01-27          C449                 C44   
8  10000009   10000009        2016-06-23          C449                 C44   
9  10000010   10000010        2014-12-05          C443                 C44   

  MORPH_ICD10_O2 BEHAVIOUR_ICD10_O2 T_BEST N_BEST M_BEST  ...  \
0         8070.0                  3      1   None   None  ...   
1         8

In [14]:
#cohort selection
query4 = """
SELECT 
    COUNT(*) AS Cohort_Count
    FROM (
    SELECT PATIENTID,TUMOURID,DIAGNOSISDATEBEST,AGE,SEX
    FROM av_tumour
    WHERE SITE_ICD10_O2_3CHAR like 'C50%'
    AND DIAGNOSISDATEBEST BETWEEN '2010-01-01' AND '2020-12-31'
    AND AGE >= 18
) AS Cohort;
"""
result4 = pql.sqldf(query4, locals())
print(result4)

   Cohort_Count
0        226379


In [15]:
sact_outcomes = pd.read_csv(r'data\sim_sact_outcome.csv')
sact_regimens = pd.read_csv(r'data\sim_sact_regimen.csv')
sact_cycle = pd.read_csv(r'data\sim_sact_cycle.csv')
sact_tumour = pd.read_csv(r'data\sim_sact_tumour.csv')
sact_drug_detail = pd.read_csv(r'data\sim_sact_drug_detail.csv')

C:\Users\sahal\AppData\Local\Temp\ipykernel_13348\3499063487.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  sact_cycle = pd.read_csv(r'data\sim_sact_cycle.csv')


In [16]:
querye3 = """
SELECT * from sact_regimens
limit 10;
"""
resulte3 = pql.sqldf(querye3, locals())
print(resulte3)

   MERGED_REGIMEN_ID  MERGED_TUMOUR_ID  HEIGHT_AT_START_OF_REGIMEN  \
0           10000001          10000001                        1.72   
1           10000002          10000002                        1.77   
2           10000003          10000003                        1.70   
3           10000004          10000004                        1.58   
4           10000005          10000005                        1.71   
5           10000006          10000006                        1.85   
6           10000007          10000007                        1.67   
7           10000008          10000007                        1.66   
8           10000009          10000008                        1.79   
9           10000010          10000008                        1.75   

   WEIGHT_AT_START_OF_REGIMEN INTENT_OF_TREATMENT DATE_DECISION_TO_TREAT  \
0                        71.0                   C             2015-10-13   
1                        68.4                   P             2014-07-01   
2

In [17]:
querye4 = """
SELECT * from sact_cycle
limit 10;
"""
resulte4 = pql.sqldf(querye4, locals()) 
print(resulte4)

   MERGED_CYCLE_ID  MERGED_REGIMEN_ID  CYCLE_NUMBER START_DATE_OF_CYCLE  \
0         10000001           10000001             1          2015-10-14   
1         10000002           10000002             1          2015-01-12   
2         10000003           10000003             1          2015-11-18   
3         10000004           10000004             1          2016-11-18   
4         10000005           10000005             1          2017-02-25   
5         10000006           10000006             2          2015-09-11   
6         10000007           10000007             1          2016-10-09   
7         10000008           10000008             1          2015-05-06   
8         10000009           10000009             1          2016-10-27   
9         10000010           10000010             1          2014-05-16   

  OPCS_PROCUREMENT_CODE PERF_STATUS_START_OF_CYCLE  MERGED_PATIENT_ID  \
0                  None                       None           10007761   
1                  X711     

In [ ]:
#Drug administered/cycles administered
query5 =""" 
SELECT
    BP.PATIENTID,
    SR.MERGED_REGIMEN_ID,
    SR.MAPPED_REGIMEN,
    SR.START_DATE_OF_REGIMEN,
    COUNT(SC.MERGED_CYCLE_ID) AS CYCLE_COUNT
FROM (
    SELECT *
    FROM av_tumour
    WHERE SITE_ICD10_O2_3CHAR like 'C50%'
      AND DIAGNOSISDATEBEST BETWEEN '2010-01-01' AND '2020-12-31'
      AND AGE >= 18
) AS BP
JOIN sact_patients AS SP ON BP.PATIENTID = SP.MERGED_PATIENT_ID
JOIN sact_tumour AS ST ON BP.PATIENTID = ST.MERGED_PATIENT_ID
JOIN sact_regimens AS SR ON ST.MERGED_TUMOUR_ID = SR.MERGED_TUMOUR_ID
LEFT JOIN sact_cycle AS SC ON SR.MERGED_REGIMEN_ID = SC.MERGED_REGIMEN_ID
GROUP BY BP.PATIENTID, SR.MERGED_REGIMEN_ID, SR.MAPPED_REGIMEN, SR.START_DATE_OF_REGIMEN
ORDER BY BP.PATIENTID, SR.START_DATE_OF_REGIMEN;
""" 
result5 = pql.sqldf(query5, locals())   
print(result5)


        PATIENTID  MERGED_REGIMEN_ID                          MAPPED_REGIMEN  \
0        10331133           10005191                Capecitabine + Mitomycin   
1        10331857           10005193                  Mitomycin Intravesical   
2        10334435           10001163                            Capecitabine   
3        10334435           10001164                        CHOP R - 21 days   
4        10334435           10006038    Rituximab (SUBCUTANEOUS Maintenance)   
...           ...                ...                                     ...   
153335  330020797          330011074                        CYCLOPHOSPHAMIDE   
153336  330020797          330012034  GEMCITABINE + OXALIPLATIN + PACLITAXEL   
153337  330024216          330006128                                    VeIP   
153338  330024216          330006130                        CHOP R - 21 days   
153339  330024216          330006129             Methotrexate + Folinic acid   

       START_DATE_OF_REGIMEN  CYCLE_COU

In [ ]:

query6 = """

SELECT
    BCT.PATIENTID,
    BCT.MERGED_REGIMEN_ID,
    BCT.MAPPED_REGIMEN,
    BCT.START_DATE_OF_REGIMEN,
    BCT.MERGED_CYCLE_ID,
    BCT.CYCLE_NUMBER,
    BCT.CYCLE_START_DATE,
    LEAD(BCT.CYCLE_START_DATE) OVER (
        PARTITION BY BCT.PATIENTID, BCT.MERGED_REGIMEN_ID 
        ORDER BY BCT.CYCLE_START_DATE
    ) AS NEXT_CYCLE_START_DATE,
    DATEDIFF(day, BCT.CYCLE_START_DATE, 
        LEAD(BCT.CYCLE_START_DATE) OVER (PARTITION BY BCT.PATIENTID, BCT.MERGED_REGIMEN_ID 
        ORDER BY BCT.CYCLE_START_DATE
        )
    ) AS DAYS_TO_NEXT_CYCLE
FROM  (SELECT
        BP.PATIENTID,
        SR.MERGED_REGIMEN_ID,
        SR.MAPPED_REGIMEN,
        SR.START_DATE_OF_REGIMEN,
        SC.MERGED_CYCLE_ID,
        SC.START_DATE_OF_CYCLE AS CYCLE_START_DATE,
        SC.CYCLE_NUMBER
    FROM (
        SELECT *
        FROM av_tumour
        WHERE SITE_ICD10_O2_3CHAR like 'C50%'
          AND DIAGNOSISDATEBEST BETWEEN '2010-01-01' AND '2020-12-31'
          AND AGE >= 18
    ) AS BP
    JOIN sact_patients AS SP ON BP.PATIENTID = SP.MERGED_PATIENT_ID
    JOIN sact_tumour AS ST ON BP.PATIENTID = ST.MERGED_PATIENT_ID
    JOIN sact_regimens AS SR ON ST.MERGED_TUMOUR_ID = SR.MERGED_TUMOUR_ID
    JOIN sact_cycle AS SC ON SR.MERGED_REGIMEN_ID = SC.MERGED_REGIMEN_ID)
 As BCT
ORDER BY BCT.PATIENTID, BCT.MERGED_REGIMEN_ID, BCT.CYCLE_START_DATE;
"""
result6 = pql.sqldf(query6, locals())
print(result6)

PandaSQLException: (sqlite3.OperationalError) no such column: day
[SQL: 

SELECT
    BCT.PATIENTID,
    BCT.MERGED_REGIMEN_ID,
    BCT.MAPPED_REGIMEN,
    BCT.START_DATE_OF_REGIMEN,
    BCT.MERGED_CYCLE_ID,
    BCT.CYCLE_NUMBER,
    BCT.CYCLE_START_DATE,
    LEAD(BCT.CYCLE_START_DATE) OVER (
        PARTITION BY BCT.PATIENTID, BCT.MERGED_REGIMEN_ID 
        ORDER BY BCT.CYCLE_START_DATE
    ) AS NEXT_CYCLE_START_DATE,
    DATEDIFF(day, BCT.CYCLE_START_DATE, 
        LEAD(BCT.CYCLE_START_DATE) OVER (PARTITION BY BCT.PATIENTID, BCT.MERGED_REGIMEN_ID 
        ORDER BY BCT.CYCLE_START_DATE
        )
    ) AS DAYS_TO_NEXT_CYCLE
FROM  (SELECT
        BP.PATIENTID,
        SR.MERGED_REGIMEN_ID,
        SR.MAPPED_REGIMEN,
        SR.START_DATE_OF_REGIMEN,
        SC.MERGED_CYCLE_ID,
        SC.START_DATE_OF_CYCLE AS CYCLE_START_DATE,
        SC.CYCLE_NUMBER
    FROM (
        SELECT *
        FROM av_tumour
        WHERE SITE_ICD10_O2_3CHAR like 'C50%'
          AND DIAGNOSISDATEBEST BETWEEN '

In [ ]:
#To find the date for difference of next cycle
#tailored the code to suit the date function in sql-lite

query7 = """
SELECT
    BCT.PATIENTID,
    BCT.MERGED_REGIMEN_ID,
    BCT.MAPPED_REGIMEN,
    BCT.START_DATE_OF_REGIMEN,
    BCT.MERGED_CYCLE_ID,
    BCT.CYCLE_NUMBER,
    BCT.CYCLE_START_DATE,
    LEAD(BCT.CYCLE_START_DATE) OVER (
        PARTITION BY BCT.PATIENTID, BCT.MERGED_REGIMEN_ID 
        ORDER BY BCT.CYCLE_START_DATE
    ) AS NEXT_CYCLE_START_DATE,
    CAST((julianday(LEAD(BCT.CYCLE_START_DATE) OVER (
        PARTITION BY BCT.PATIENTID, BCT.MERGED_REGIMEN_ID 
        ORDER BY BCT.CYCLE_START_DATE
    )) - julianday(BCT.CYCLE_START_DATE)) AS INTEGER) AS DAYS_TO_NEXT_CYCLE
FROM  (SELECT
        BP.PATIENTID,
        SR.MERGED_REGIMEN_ID,
        SR.MAPPED_REGIMEN,
        SR.START_DATE_OF_REGIMEN,
        SC.MERGED_CYCLE_ID,
        SC.START_DATE_OF_CYCLE AS CYCLE_START_DATE,
        SC.CYCLE_NUMBER
    FROM (
        SELECT *
        FROM av_tumour
        WHERE SITE_ICD10_O2_3CHAR like 'C50%'
          AND DIAGNOSISDATEBEST BETWEEN '2010-01-01' AND '2020-12-31'
          AND AGE >= 18
    ) AS BP
    JOIN sact_patients AS SP ON BP.PATIENTID = SP.MERGED_PATIENT_ID
    JOIN sact_tumour AS ST ON BP.PATIENTID = ST.MERGED_PATIENT_ID
    JOIN sact_regimens AS SR ON ST.MERGED_TUMOUR_ID = SR.MERGED_TUMOUR_ID
    JOIN sact_cycle AS SC ON SR.MERGED_REGIMEN_ID = SC.MERGED_REGIMEN_ID)
 As BCT
ORDER BY BCT.PATIENTID, BCT.MERGED_REGIMEN_ID, BCT.CYCLE_START_DATE;
"""
result7 = pql.sqldf(query7, locals())
print(result7)  # Changed from result6 to result7


        PATIENTID  MERGED_REGIMEN_ID               MAPPED_REGIMEN  \
0        10331133           10005191     Capecitabine + Mitomycin   
1        10331857           10005193       Mitomycin Intravesical   
2        10331857           10005193       Mitomycin Intravesical   
3        10331857           10005193       Mitomycin Intravesical   
4        10331857           10005193       Mitomycin Intravesical   
...           ...                ...                          ...   
623031  330024216          330006129  Methotrexate + Folinic acid   
623032  330024216          330006129  Methotrexate + Folinic acid   
623033  330024216          330006129  Methotrexate + Folinic acid   
623034  330024216          330006130             CHOP R - 21 days   
623035  330024216          330006130             CHOP R - 21 days   

       START_DATE_OF_REGIMEN  MERGED_CYCLE_ID  CYCLE_NUMBER CYCLE_START_DATE  \
0                 2015-07-09         10005191             1       2015-07-09   
1          

In [27]:
queryex1 = """
select * from sact_outcomes
limit 10;
""" 
resultex1 = pql.sqldf(queryex1, locals())
print(resultex1)

   MERGED_OUTCOME_ID  MERGED_REGIMEN_ID DATE_OF_FINAL_TREATMENT  \
0           10000001           10000001                    None   
1           10000002           10000002                    None   
2           10000003           10000003                    None   
3           10000004           10000004              2017-01-28   
4           10000005           10000005                    None   
5           10000006           10000006                    None   
6           10000007           10000007              2017-03-22   
7           10000008           10000008                    None   
8           10000009           10000009                    None   
9           10000010           10000010                    None   

  REGIMEN_MOD_DOSE_REDUCTION REGIMEN_MOD_TIME_DELAY REGIMEN_MOD_STOPPED_EARLY  \
0                          N                      Y                         N   
1                          N                   None                         N   
2                  

In [35]:
querys1 = """
SELECT AP.PATIENTID,AP.NEWVITALSTATUS,AT.DIAGNOSISDATEBEST,AT.STAGE_BEST
FROM av_patients AS AP
JOIN av_tumour AS AT ON AP.PATIENTID = AT.PATIENTID
WHERE SITE_ICD10_O2_3CHAR like 'C50%'
AND DIAGNOSISDATEBEST BETWEEN '2010-01-01' AND '2020-12-31'
AND AGE >= 18
AND NEWVITALSTATUS IS NOT NULL
;
"""
results1 = pql.sqldf(querys1, locals())
print(results1)

        PATIENTID NEWVITALSTATUS DIAGNOSISDATEBEST STAGE_BEST
0        10339996              A        2017-05-16         2B
1        10339999              A        2015-05-18         1A
2        10340011              A        2016-11-15         2A
3        10340014              A        2016-03-19          ?
4        10340015              A        2016-10-30         2A
...           ...            ...               ...        ...
226374  330024216              D        2014-10-28         1A
226375  330006741              A        2015-04-17          ?
226376  330016564              A        2014-02-23          ?
226377  330021447              A        2017-01-01          1
226378  330022940              A        2017-09-12          3

[226379 rows x 4 columns]


In [39]:
resultskm=results1.to_csv(r'data\km4_stage.csv', index=False)

In [ ]:
"""# Load required packages
library(survival)
library(survminer)

# Read the exported CSV file
data <- read.csv(r'data\km4_stage.csv')

# Data preparation
data$event <- ifelse(data$NEWVITALSTATUS == "D", 1, 0)  # Assuming "D" means death
data$time <- as.numeric(difftime(
  as.Date(data$VITALSTATUSDATE, format="%Y-%m-%d"),
  as.Date(data$DIAGNOSISDATEBEST, format="%Y-%m-%d"), 
  units="days"
)) / 30.44  # Convert to months

# Create and fit survival model stratified by stage
km_fit_by_stage <- survfit(Surv(time, event) ~ STAGE_BEST, data = data)

# Plot the stratified Kaplan-Meier curves
ggsurvplot(
  km_fit_by_stage,
  data = data,
  risk.table = TRUE,
  pval = TRUE,           # Add log-rank p-value
  conf.int = TRUE,
  palette = "jco",       #

  """


SyntaxError: invalid syntax (2182166325.py, line 9)